In [1]:
from collections import defaultdict, Counter 
import colorsys
import itertools
import pickle 
import os
import shutil
import subprocess

import fcsparser
import fcswrite
import numpy as np
import pandas as pd
import networkx as nx 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.backends.backend_pdf import PdfPages
from PIL import Image
from PyPDF2 import PdfFileWriter, PdfFileReader, PdfFileMerger
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from IPython.display import display, HTML

import ot

from scipy.spatial import distance, ConvexHull
from scipy.stats import spearmanr

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score
from sklearn.manifold import TSNE
import fcsparser 
from ete3 import Tree, TreeNode, TreeStyle, TextFace, add_face_to_node

from utils import *

data_dir = '.'

## Split Supplementary Data into FCS Files 

In [10]:
supplementary_data = pd.read_csv('../Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')
marker_cols = list(supplementary_data.columns[1:30])
supplementary_data = supplementary_data[marker_cols + ['Imaging phenotype cluster ID']]
ids_to_names = pd.read_csv('ClusterIDtoName.txt', sep='\t')
cell_lines = list(ids_to_names['ID'].values)
ids_to_names = dict(zip(ids_to_names['ID'].values, ids_to_names['Name'].values))
# remove dirt from supplementary data 
supplementary_annotations = pd.read_excel('../Suppl.Table2.cluster annotations and cell counts.xlsx')
exclude = supplementary_annotations.loc[
                                     supplementary_annotations['Imaging phenotype (cell type)'] == 'dirt',
                                     'X-shift cluster ID']
supplementary_data = supplementary_data[~supplementary_data['Imaging phenotype cluster ID'].isin(exclude)]
supplementary_data['Imaging phenotype cluster ID'] = supplementary_data['Imaging phenotype cluster ID'].apply(lambda x: ids_to_names[x].replace('/', '.'))
suppl_groupby = supplementary_data.groupby('Imaging phenotype cluster ID')
if not os.path.isdir('scaffold_landmarks'):
    os.mkdir('scaffold_landmarks')
for cell_id, data in suppl_groupby:
    print('Writing data for cell type', cell_id)
    data.drop('Imaging phenotype cluster ID', axis=1, inplace=True)
    fcswrite.write_fcs(filename='scaffold_landmarks/cell_{}.fcs'.format(cell_id), 
                       chn_names=list(data.columns), 
                       data=data.values)

Writing data for cell type B cells


/home/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Writing data for cell type B220(+) DN T cells
Writing data for cell type CD106(+)CD16.32(+)CD31(+) stroma
Writing data for cell type CD106(+)CD16.32(+)CD31(-)Ly6C(-) stroma
Writing data for cell type CD106(+)CD16.32(-)Ly6C(+)CD31(+)
Writing data for cell type CD106(-)CD16.32(+)Ly6C(+)CD31(-)
Writing data for cell type CD106(-)CD16.32(-)Ly6C(+)CD31(+) stroma
Writing data for cell type CD11c(+) B cells
Writing data for cell type CD3(+) other markers (-)
Writing data for cell type CD31(hi) vascular
Writing data for cell type CD4(+) T cells
Writing data for cell type CD4(+)CD8(-)cDC
Writing data for cell type CD4(+)MHCII(+)
Writing data for cell type CD4(-)CD8(+)cDC
Writing data for cell type CD4(-)CD8(-) cDC
Writing data for cell type CD8(+) T cells
Writing data for cell type ERTR7(+) stroma
Writing data for cell type F4.80(+) mphs
Writing data for cell type FDCs
Writing data for cell type NK cells
Writing data for cell type capsule
Writing data for cell type erythroblasts
Writing data fo

## Run Vite on Cell Data 

In [2]:
# write clusters of tree to FCS files 
tree = pickle.load(open('tree_combined_for_html.pkl', 'rb'))
supplementary_data = pd.read_csv('../Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')
marker_cols = list(supplementary_data.columns[1:30])
supplementary_data.rename(columns={'X.X': 'X', 'Y.Y': 'Y', 'Z.Z': 'Z'}, inplace=True)
supplementary_data['CD45_int'] = supplementary_data['CD45'].astype(int)
ids_to_names = pd.read_csv('ClusterIDtoName.txt', sep='\t')
cell_lines = list(ids_to_names['ID'].values)
ids_to_names = dict(zip(ids_to_names['ID'].values, ids_to_names['Name'].values))
# remove dirt from supplementary data 
supplementary_annotations = pd.read_excel('../Suppl.Table2.cluster annotations and cell counts.xlsx')
dirt = supplementary_annotations.loc[supplementary_annotations['Imaging phenotype (cell type)'] == 'dirt', 
                                     'X-shift cluster ID']
supplementary_data = supplementary_data[~supplementary_data['Imaging phenotype cluster ID'].isin(dirt)]
supplementary_data['sample'] = supplementary_data['sample_Xtile_Ytile'].apply(lambda x: x.split('_')[0])
supplementary_data['Imaging phenotype cluster ID'] = supplementary_data['Imaging phenotype cluster ID'].apply(lambda x: ids_to_names[x].replace('/', '.'))
suppl_converted = convert_coordinates(supplementary_data)[['X', 'Y', 'Z', 'sample', 'Imaging phenotype cluster ID']
                                                          + marker_cols]
del supplementary_data
print(marker_cols)

['CD45', 'Ly6C', 'TCR', 'Ly6G', 'CD19', 'CD169', 'CD106', 'CD3', 'CD1632', 'CD8a', 'CD90', 'F480', 'CD11c', 'Ter119', 'CD11b', 'IgD', 'CD27', 'CD5', 'CD79b', 'CD71', 'CD31', 'CD4', 'IgM', 'B220', 'ERTR7', 'CD35', 'CD2135', 'CD44', 'NKp46']


In [21]:
def write_layers(tree):
    layers = get_layers(tree)
    for layer_ind, layer in enumerate(layers):
        print('Writing layer', layer_ind)
        layer_dir = 'scaffold_analysis/Layer_' + str(layer_ind)
        if not os.path.exists(layer_dir):
            os.makedirs(layer_dir)
        all_clusters_markers = []
        for cluster_ind, node in enumerate(layer):
            node_markers= pd.merge(node.coords, suppl_converted, how='inner', on=['X', 'Y', 'Z', 'sample'])
            node_markers = node_markers[marker_cols].mean()
            all_clusters_markers.append(node_markers)
        all_clusters_markers = pd.DataFrame(all_clusters_markers)
        all_clusters_markers[all_clusters_markers < 0] = 0
        all_clusters_markers['cellType'] = ['Cluster_' + str(i) for i in range(all_clusters_markers.shape[0])]
        all_clusters_markers.to_csv(layer_dir + '/clusters_avg_markers.txt', sep='\t', index=False)
            
write_layers(tree)

Writing layer 0
Writing layer 1
Writing layer 2
Writing layer 3
Writing layer 4
Writing layer 5
Writing layer 6
Writing layer 7
Writing layer 8
Writing layer 9
Writing layer 10
Writing layer 11
Writing layer 12
Writing layer 13
Writing layer 14
Writing layer 15
Writing layer 16
Writing layer 17
Writing layer 18
Writing layer 19
Writing layer 20
Writing layer 21
Writing layer 22
Writing layer 23
Writing layer 24
Writing layer 25
Writing layer 26
Writing layer 27
Writing layer 28
Writing layer 29
Writing layer 30
Writing layer 31
Writing layer 32
Writing layer 33
Writing layer 34
Writing layer 35
Writing layer 36
Writing layer 37
Writing layer 38
Writing layer 39
Writing layer 40
Writing layer 41


In [9]:
def match_clusters_to_types(tree, save=True):
    ts = TreeStyle()
    ts.show_leaf_name = False
    ts.rotation = 90
    def my_layout(node):
        F = TextFace(node.name, tight_text=True)
        F.rotation = 270
        add_face_to_node(F, node, column=0, position="branch-right")
        F.border.width = 1
        F.inner_border.width = 1
    ts.layout_fn = my_layout
    
    annotated_tree = recreate_tree(tree, color=False)
    layers = get_layers(annotated_tree)
    for layer_ind, layer in enumerate(layers):
        graph_filename = 'scaffold_result/Layer_{}/clusters_avg_markers.txt.graphml'.format(layer_ind)
        graph = nx.read_graphml(graph_filename)
        nodes_clusters = [(node, data) for node, data in graph.nodes(data=True) if data['cellType'].startswith('Cluster_')]
        converter = {node: data['cellType'] for node, data in graph.nodes(data=True)}
        nodes_landmark = set([node for node, data in graph.nodes(data=True) if not data['cellType'].startswith('Cluster_')])
        cluster_types = {}
        for node, data in nodes_clusters:
            max_weight_node, max_weight_val = None, None
            for neighbor, edge_data in graph[node].items():
                if neighbor in nodes_landmark:
                    if max_weight_val is None or edge_data['weight'] > max_weight_val:
                        max_weight_node = neighbor
                        max_weight_val = edge_data['weight']
            cluster_types[converter[node]] = converter[max_weight_node]
        for ind, node in enumerate(layer):
            node.add_features(name=cluster_types['Cluster_' + str(ind)])
    if save:       
        annotated_tree.render('matched_tree.pdf', tree_style=ts)
    else:
        return annotated_tree.render('%%inline', tree_style=ts)
    
match_clusters_to_types(tree)

In [10]:
def get_layer_intersection(layer):
    # for layer of tree, get cell types that each nodes intersects with most 
    most_intersect = {}
    for ind, node in enumerate(layer):
        overlap = pd.merge(node.coords, suppl_converted, on=['X', 'Y', 'Z', 'sample'], how='inner')
        counts = overlap['Imaging phenotype cluster ID'].value_counts().to_dict()
        most_intersect_node = max(counts, key=counts.get)
        most_intersect['Cluster_' + str(ind)] = most_intersect_node
        
    return most_intersect 

def max_overlap_to_types(tree, save=True):
    ts = TreeStyle()
    ts.show_leaf_name = False
    ts.rotation = 90
    def my_layout(node):
        F = TextFace(node.name, tight_text=True)
        F.rotation = 270
        add_face_to_node(F, node, column=0, position="branch-right")
        F.border.width = 1
        F.inner_border.width = 1
    ts.layout_fn = my_layout
    
    annotated_tree = recreate_tree(tree, color=False)
    orig_layers = get_layers(tree)
    new_layers = list(get_layers(annotated_tree))   
    for layer_ind, layer in enumerate(orig_layers):
        cluster_types = get_layer_intersection(layer)
        for ind, node in enumerate(new_layers[layer_ind]):
            node.add_features(name=cluster_types['Cluster_' + str(ind)])
            
    if save:       
        annotated_tree.render('overlap_labeled_tree.pdf', tree_style=ts)
    else:
        return annotated_tree.render('%%inline', tree_style=ts)
    
max_overlap_to_types(tree)